# Cloud Native Satellite Data Demo 2
* Generated by Gemini (Pro 2.5)
* Prompt said to use pystac_client, odc.stac, hvplot and panel

In [ ]:
import panel as pn
import pystac_client
import planetary_computer
import odc.stac
import rioxarray
import hvplot.xarray
import numpy as np

# --- 1. Panel Setup and App Title ---
# Load the panel extension to make plots interactive
pn.extension()

# Create a title and description for our app
title = pn.pane.Markdown("## 🛰️ Cloud-Native Satellite Data Explorer", sizing_mode='stretch_width')
description = pn.pane.Markdown("""
Select a date from the dropdown below to view a true-color Sentinel-2 satellite image.
The map is fully interactive—you can pan and zoom. The data is loaded directly from the cloud!
""", sizing_mode='stretch_width')

# --- 2. One-Time Data Loading ---
# This section runs only once when the app starts.
@pn.cache
def load_data():
    """
    Finds and loads satellite data using pystac and odc-stac.
    The pn.cache decorator prevents this from re-running on every interaction.
    """
    print("Searching for satellite imagery...")
    # Define our area of interest (AOI) - a rough box around Cape Cod
    bbox = [-71.0, 41.5, -69.8, 42.2]

    # Connect to the Planetary Computer STAC API
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )

    # Search for Sentinel-2 Level-2A data with minimal cloud cover.
    # We use a recent, complete summer for good data (adjust if needed).
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime="2025-08-01/2025-10-30",
        query={"eo:cloud_cover": {"lt": 10}},
    )
    items = search.item_collection()
    if not items:
        return None, None # Handle case with no items found

    print(f"Found {len(items)} scenes. Loading data cube...")
    # Load the data using odc-stac
    data_ds = odc.stac.stac_load(
        items,
        bands=["B04", "B03", "B02"], # Red, Green, Blue
        bbox=bbox,
        resolution=100, # Use a coarser resolution for faster app performance
        chunks={"x": 2048, "y": 2048},
    )

    # Attach the CRS information
    source_crs = 'EPSG:32619'
    data_ds = data_ds.rio.write_crs(source_crs)
    
    print("Data loading complete.")
    return data_ds, source_crs

# Execute the data loading function
data_cube, source_crs = load_data()

# --- 3. Create Interactive Widgets ---
if data_cube is not None:
    # Get the available dates and format them nicely for the dropdown
    date_options = [np.datetime_as_string(d, unit='D') for d in data_cube.time.values]
    date_selector = pn.widgets.Select(
        name='Scene Date',
        options=date_options,
        value=date_options[0] # Default to the first date
    )
else:
    # If no data is found, just show a message.
    date_selector = pn.pane.Markdown("### No data found for the specified criteria.")


# --- 4. Define the Plotting Function ---
def create_map(selected_date):
    """
    Generates an hvplot map for a given date.
    This function will be re-run whenever the selected_date changes.
    """
    if data_cube is None:
        return pn.pane.Markdown("Could not load data to generate map.")

    print(f"Generating map for {selected_date}...")
    
    # Select the data for the chosen time
    scene = data_cube.sel(time=selected_date)

    # Prepare and scale the RGB bands
    rgb = scene[["B04", "B03", "B02"]].to_array(dim="band")
    rgb_scaled = rgb.clip(0, 3000) / 3000

    # Generate the interactive plot, telling hvplot the source CRS
    image_plot = rgb_scaled.squeeze().hvplot.rgb(
        x='x',
        y='y',
        bands='band',
        crs=source_crs,
        geo=True,
        tiles='OSM',
        rasterize=True,
        frame_width=700,
        project=True,
        title=f"{selected_date}"
    )
    return image_plot

# --- 5. Bind the Widget to the Plotting Function ---
# pn.bind links the 'selected_date' argument of create_map to the 'value' of date_selector
interactive_map = pn.bind(create_map, selected_date=date_selector)

# --- 6. Assemble the App Layout ---
# The final app is a column containing the title, description, widget, and the interactive map
app_layout = pn.Column(
    title,
    description,
    date_selector,
    # Panel will automatically show a loading indicator for the interactive map
    interactive_map,
    sizing_mode='stretch_width'
)


# --- 7. Make the App Serveable ---
# This allows the app to be displayed in a browser when run with "panel serve"
app_layout.servable()

